In [ ]:
#Cell1: Install required libraries
!pip -q install scikit-learn pandas matplotlib

#Cell1: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import random
np.random.seed(42)
random.seed(42)

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
#Cell2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Cell2: Path configuration
#IMPORTANT: Change this path to where autos.csv is stored in YOUR Drive
DATA_PATH = "/content/drive/MyDrive/autos.csv"

#Cell2: Load dataset
#Note: autos.csv uses latin-1 encoding (common for this dataset)
autos_raw = pd.read_csv(DATA_PATH, encoding="latin1")

print("autos.csv loaded from Google Drive")
print("Shape (rows, columns):", autos_raw.shape)
print("\nColumns:")
print(autos_raw.columns.tolist())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
autos.csv loaded from Google Drive
Shape (rows, columns): (371528, 20)

Columns:
['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType', 'brand', 'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen']


In [ ]:
#Cell3: Basic dataset info & 10 sample records (RAW)
print("=== DataFrame .info() ===")
autos_raw.info()

print("\n=== First 10 records (RAW dataset) ===")
sample_raw = autos_raw.head(10).copy()
display(sample_raw)

#Keep a copy of these exact 10 rows by index for later comparison
sample_raw_indices = sample_raw.index.to_list()
print("\nStored indices of the 10 sample records:", sample_raw_indices)

=== DataFrame .info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          371528 non-null  object
 1   name                 371528 non-null  object
 2   seller               371528 non-null  object
 3   offerType            371528 non-null  object
 4   price                371528 non-null  int64 
 5   abtest               371528 non-null  object
 6   vehicleType          333659 non-null  object
 7   yearOfRegistration   371528 non-null  int64 
 8   gearbox              351319 non-null  object
 9   powerPS              371528 non-null  int64 
 10  model                351044 non-null  object
 11  kilometer            371528 non-null  int64 
 12  monthOfRegistration  371528 non-null  int64 
 13  fuelType             338142 non-null  object
 14  brand                371528 non-null  object
 15  notRepai

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...,privat,Angebot,650,test,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,Peugeot_206_CC_110_Platinum,privat,Angebot,2200,test,cabrio,2004,manuell,109,2_reihe,150000,8,benzin,peugeot,nein,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,VW_Derby_Bj_80__Scheunenfund,privat,Angebot,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,Ford_C___Max_Titanium_1_0_L_EcoBoost,privat,Angebot,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,VW_Golf_4_5_tuerig_zu_verkaufen_mit_Anhaengerk...,privat,Angebot,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06



Stored indices of the 10 sample records: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
#Cell4: Dataset analysis (RAW)

#Number of records and attributes
num_rows, num_cols = autos_raw.shape
print(f"Number of records (rows): {num_rows}")
print(f"Number of attributes (columns): {num_cols}")

#Missing values per column
print("\n=== Missing values per column (RAW) ===")
missing_raw = autos_raw.isna().sum().sort_values(ascending=False)
display(missing_raw)

#Basic stats for numeric columns
print("\n=== Numeric columns descriptive statistics (RAW) ===")
display(autos_raw.describe())

#Basic overview of categorical columns (some examples)
categorical_cols = autos_raw.select_dtypes(include=["object"]).columns.tolist()
print("\nNumber of categorical columns:", len(categorical_cols))
print("Example categorical columns:", categorical_cols[:10])

#Unique counts for some key categorical attributes
for col in ["brand", "vehicleType", "gearbox", "fuelType", "notRepairedDamage"]:
    if col in autos_raw.columns:
        print(f"\n=== Value counts for {col} (top 10) ===")
        display(autos_raw[col].value_counts(dropna=False).head(10))

Number of records (rows): 371528
Number of attributes (columns): 20

=== Missing values per column (RAW) ===


,0
notRepairedDamage,72060
vehicleType,37869
fuelType,33386
model,20484
gearbox,20209
seller,0
name,0
dateCrawled,0
offerType,0
abtest,0



=== Numeric columns descriptive statistics (RAW) ===


,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration,nrOfPictures,postalCode
count,3.715280e+05,371528.000000,371528.000000,371528.000000,371528.000000,371528.0,371528.00000
mean,1.729514e+04,2004.577997,115.549477,125618.688228,5.734445,0.0,50820.66764
std,3.587954e+06,92.866598,192.139578,40112.337051,3.712412,0.0,25799.08247
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.00000
25%,1.150000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30459.00000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49610.00000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71546.00000
max,2.147484e+09,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.00000



Number of categorical columns: 13
Example categorical columns: ['dateCrawled', 'name', 'seller', 'offerType', 'abtest', 'vehicleType', 'gearbox', 'model', 'fuelType', 'brand']

=== Value counts for brand (top 10) ===


...

,count
gearbox,
manuell,274214
automatik,77105
NaN,20209



=== Value counts for fuelType (top 10) ===


,count
fuelType,
benzin,223857
diesel,107746
NaN,33386
lpg,5378
cng,571
hybrid,278
andere,208
elektro,104



=== Value counts for notRepairedDamage (top 10) ===


,count
notRepairedDamage,
nein,263182
NaN,72060
ja,36286


In [ ]:
#Cell5: Focused quality checks for key numeric fields

important_numeric = ["yearOfRegistration", "price", "powerPS", "kilometer"]
print("=== Descriptive stats for key numeric fields (RAW) ===")
display(autos_raw[important_numeric].describe())

#Check min/max values (useful to justify cleaning thresholds)
print("\n=== Min/Max of key numeric fields ===")
for col in important_numeric:
    print(f"{col}: min={autos_raw[col].min()}, max={autos_raw[col].max()}")

#Check for duplicate rows
num_duplicates = autos_raw.duplicated().sum()
print(f"\nNumber of fully duplicated rows: {num_duplicates}")

=== Descriptive stats for key numeric fields (RAW) ===


,yearOfRegistration,price,powerPS,kilometer
count,371528.000000,3.715280e+05,371528.000000,371528.000000
mean,2004.577997,1.729514e+04,115.549477,125618.688228
std,92.866598,3.587954e+06,192.139578,40112.337051
min,1000.000000,0.000000e+00,0.000000,5000.000000
25%,1999.000000,1.150000e+03,70.000000,125000.000000
50%,2003.000000,2.950000e+03,105.000000,150000.000000
75%,2008.000000,7.200000e+03,150.000000,150000.000000
max,9999.000000,2.147484e+09,20000.000000,150000.000000



=== Min/Max of key numeric fields ===
yearOfRegistration: min=1000, max=9999
price: min=0, max=2147483647
powerPS: min=0, max=20000
kilometer: min=5000, max=150000

Number of fully duplicated rows: 4


In [ ]:
#Cell6: Data cleaning & preprocessing

autos = autos_raw.copy()

print("Initial number of rows:", len(autos))

#Step1: Remove fully duplicated rows
before = len(autos)
autos = autos.drop_duplicates()
after = len(autos)
print(f"Step1 - Drop duplicates: {before} -> {after} rows")

#Step2: Filter unrealistic registration years
#We keep cars registered between 1950 and 2016 (dataset is from 2016)
before = len(autos)
autos = autos[(autos["yearOfRegistration"] >= 1950) & (autos["yearOfRegistration"] <= 2016)]
after = len(autos)
print(f"Step2 - Filter yearOfRegistration (1950-2016): {before} -> {after} rows")

#Step3: Filter unrealistic prices
#Keep cars with price between 100 and 100000 (Euro). This removes zeros and extreme outliers.
before = len(autos)
autos = autos[(autos["price"] >= 100) & (autos["price"] <= 100000)]
after = len(autos)
print(f"Step3 - Filter price (100 to 100000): {before} -> {after} rows")

#Step4: Filter unrealistic powerPS values
#Keep cars with power between 10 PS and 500 PS (removes 0 and extreme errors)
before = len(autos)
autos = autos[(autos["powerPS"] >= 10) & (autos["powerPS"] <= 500)]
after = len(autos)
print(f"Step4 - Filter powerPS (10 to 500): {before} -> {after} rows")

#Step5: Handle missing values for modeling
#For the model, we will use the following features:
model_features = [
    "yearOfRegistration",
    "kilometer",
    "powerPS",
    "gearbox",
    "vehicleType",
    "brand",
    "fuelType",
    "notRepairedDamage"
]

#Subset to features + target
autos_model = autos[model_features + ["price"]].copy()

#Drop rows with any missing values in these columns
before = len(autos_model)
autos_model = autos_model.dropna(subset=model_features + ["price"])
after = len(autos_model)
print(f"Step5 - Drop rows with missing values in model features: {before} -> {after} rows")

print("\nFinal shape of cleaned dataset used for modeling:", autos_model.shape)

Initial number of rows: 371528
Step1 - Drop duplicates: 371528 -> 371524 rows
Step2 - Filter yearOfRegistration (1950-2016): 371524 -> 356555 rows
Step3 - Filter price (100 to 100000): 356555 -> 343791 rows
Step4 - Filter powerPS (10 to 500): 343791 -> 310862 rows
Step5 - Drop rows with missing values in model features: 310862 -> 252730 rows

Final shape of cleaned dataset used for modeling: (252730, 9)


In [ ]:
#Cell7: Show the 10 sample records AFTER CLEANING

print("Original indices of the 10 sample records (RAW):", sample_raw_indices)

#We check which of the original 10 records survived in autos_model
existing_indices = [idx for idx in sample_raw_indices if idx in autos_model.index]
print("Indices that survived in the final modeling dataset:", existing_indices)

#Retrieve the cleaned records that still exist
sample_cleaned = autos_model.loc[existing_indices].copy()

#If fewer than 10 survived, add more clean rows
if len(sample_cleaned) < 10:
    needed = 10 - len(sample_cleaned)

    #Get additional valid rows (exclude existing)
    additional_rows = autos_model.drop(index=existing_indices).head(needed)

    #Combine the survivors + new clean rows
    sample_cleaned = pd.concat([sample_cleaned, additional_rows], ignore_index=False)

print("\n=== Final 10 sample records AFTER CLEANING (from autos_model) ===")
display(sample_cleaned)

print("\nNumber of sample records after adjustment:", len(sample_cleaned))


Original indices of the 10 sample records (RAW): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Indices that survived in the final modeling dataset: [1, 3, 4, 5, 6]

=== Final 10 sample records AFTER CLEANING (from autos_model) ===


,yearOfRegistration,kilometer,powerPS,gearbox,vehicleType,brand,fuelType,notRepairedDamage,price
1,2011,125000,190,manuell,coupe,audi,diesel,ja,18300
3,2001,150000,75,manuell,kleinwagen,volkswagen,benzin,nein,1500
4,2008,90000,69,manuell,kleinwagen,skoda,diesel,nein,3600
5,1995,150000,102,manuell,limousine,bmw,benzin,ja,650
6,2004,150000,109,manuell,cabrio,peugeot,benzin,nein,2200
10,2004,150000,105,manuell,limousine,mazda,benzin,nein,2000
11,2005,150000,140,manuell,kombi,volkswagen,diesel,ja,2799
14,2011,70000,190,manuell,suv,nissan,diesel,nein,17999
17,2004,150000,75,automatik,kleinwagen,renault,benzin,nein,1750
18,2007,150000,136,manuell,bus,ford,diesel,nein,7550



Number of sample records after adjustment: 10


In [ ]:
#Cell8: Fast Random Forest Regressor

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

#Separate features (X) and target (y)
X = autos_model[model_features]
y = autos_model["price"]

#Numeric + Categorical columns
numeric_features = ["yearOfRegistration", "kilometer", "powerPS"]
categorical_features = ["gearbox", "vehicleType", "brand", "fuelType", "notRepairedDamage"]

print("Numeric:", numeric_features)
print("Categorical:", categorical_features)

#Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

#Random Forest
model = RandomForestRegressor(
    n_estimators=30,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

#Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Train size:", X_train.shape, "Test size:", X_test.shape)

#Pipeline
rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

#Train
rf_pipeline.fit(X_train, y_train)

#Predict
y_pred = rf_pipeline.predict(X_test)

#Evaluate
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n Model Performance (Random Forest)")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.3f}")

Numeric: ['yearOfRegistration', 'kilometer', 'powerPS']
Categorical: ['gearbox', 'vehicleType', 'brand', 'fuelType', 'notRepairedDamage']
Train size: (202184, 8) Test size: (50546, 8)

 Model Performance (Random Forest)
Mean Absolute Error (MAE): 1294.75
R² Score: 0.889


In [ ]:
#Cell9: Save cleaned dataset used for modeling

CLEANED_PATH = "/content/drive/MyDrive/autos_cleaned_for_model.csv"

autos_model.to_csv(CLEANED_PATH, index=False)
print(f"Cleaned dataset saved to: {CLEANED_PATH}")

Cleaned dataset saved to: /content/drive/MyDrive/autos_cleaned_for_model.csv
